In [1]:
import numpy as np

In [2]:
class NeuralNetwork:
    def initialize_prarameters(self, input_nodes, hidden_nodes):
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
    
        self.w1 = np.random.randn(hidden_nodes, input_nodes) * 0.1
        self.w2 = np.random.randn(1, hidden_nodes)* 0.1    
        
        weights = {'w1': self.w1, 'w2': self.w2}
        return weights
    
    def sigmoid(self, z, derivation=False):
        if derivation == True:
            return z*(1-z)
        
        else:
            return 1/(1+np.exp(-z))
        
    def forback_prop(self, X, y, w1, w2):
        #forward propagation
        z1 = np.dot(X, w1.T) 
        a1 = self.sigmoid(z1)
        
        z2 = np.dot(a1, w2.T)
        a2 = self.sigmoid(z2)
        
        #backward propagation
        o_error = y - a2 
        d_output = o_error * self.sigmoid(a2, derivation=True) #derivation of output layer
        self.der_output = np.dot(d_output.T, a1)

        h_error = np.dot(d_output, w2) 
        d_hidden = h_error * self.sigmoid(a1, derivation=True) #derivation of hidden layer
        self.der_hidden = np.dot(d_hidden.T, X)
        
        cost = (np.mean(np.abs(o_error)))
        
        return self.der_hidden, self.der_output, cost
        
    def update_weights(self,X_train, y_train, lr, iteration, w1, w2):
        weights = {}
        costs = []
        iterations = []
            
        for i in range(iteration):
            der_hidden, der_output, cost = self.forback_prop(X_train, y_train, w1, w2)
            #weights update
            w1 += lr * der_hidden   
            w2 += lr * der_output
            costs.append(cost)
            iterations.append(i+1)
            if i % 500 == 0:
                print(f'training epoch: {i+1}   cost: {cost}')
            
        weights['w1'] = self.w1
        weights['w2'] = self.w2
        
        return weights, costs, iterations

    def predict(self, X, weights):
        a1 = self.sigmoid(np.dot(X, weights['w1'].T))
        y_activation = self.sigmoid(np.dot(a1, weights['w2'].T))
        y_pred = np.zeros_like(y_activation)
        
        for i in range(len(y_activation)):
            if y_activation[i, 0] < 0.5:
                y_pred[i, 0] = 0
                
            else:
                y_pred[i, 0] = 1
        
        return y_pred
        
    def neural_network(self, X_train, y_train, X_test, y_test, learning_rate, iteration, hidden_nodes):
        input_nodes = X_train.shape[1]
        
        weights = self.initialize_prarameters(input_nodes, hidden_nodes)
        
        weights, costs, iterations = self.update_weights(X_train, y_train, learning_rate, iteration, weights['w1'], weights['w2'])
        
        train_pred = self.predict(X_train, weights)
        test_pred = self.predict(X_test, weights)
        
        train_score = 100 - (np.mean(np.abs(train_pred - y_train)) * 100)
        test_score = 100 - (np.mean(np.abs(test_pred - y_test)) * 100)
        print('\nTrain accuracy: ', train_score)
        print('Test accuracy: ', test_score)
        
        return costs, iterations
        